# NOTE: Code adapted to work with TensorFlow v1. Recommend to use V2 (integrated Keras). Examples only for learning purposes

# Imports

In [3]:
from numpy.random import seed
seed(888)

from tensorflow import random
random.set_seed(404)

In [592]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [6]:
X_TRAIN_PATH = "MNIST/digit_xtrain.csv"
X_TEST_PATH = "MNIST/digit_xtest.csv"
Y_TRAIN_PATH = "MNIST/digit_ytrain.csv"
Y_TEST_PATH = "MNIST/digit_ytest.csv"

LOGGING_PATH = "tensorboard_mnist_digit_logs/"

NR_CLASSES = 10

VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS


# Get the Data

In [8]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=",", dtype=int)

CPU times: total: 0 ns
Wall time: 9.2 ms


In [9]:
y_train_all.shape

(60000,)

In [10]:
%%time

y_test = np.loadtxt(Y_TEST_PATH, delimiter=",", dtype=int)

CPU times: total: 15.6 ms
Wall time: 2.43 ms


In [11]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=",", dtype=int)

CPU times: total: 4.25 s
Wall time: 4.32 s


In [12]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=",", dtype=int)

CPU times: total: 703 ms
Wall time: 726 ms


# Explore

In [14]:
x_train_all.shape

(60000, 784)

In [15]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [16]:
x_test.shape

(10000, 784)

In [17]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [19]:
# Re-Scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

#### Convert target values to one-hot encoding

In [21]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [22]:
np.eye(10)[2] #Extract third row of first array

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
values[3]

1

In [24]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [25]:
y_train_all.shape

(60000, 10)

In [26]:
y_test = np.eye(NR_CLASSES)[y_test]

In [27]:
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [29]:
x_val, x_train = x_train_all[:VALIDATION_SIZE], x_train_all[VALIDATION_SIZE:]
y_val, y_train = y_train_all[:VALIDATION_SIZE], y_train_all[VALIDATION_SIZE:]

In [30]:
x_train.shape

(50000, 784)

In [31]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [691]:
tf.compat.v1.disable_v2_behavior()

X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name="X")
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name="labels")

### Neural Network Architecture

#### Hyperparameters

In [693]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [694]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.compat.v1.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name="W")
        
        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name="B")
        
        layer_in = tf.matmul(input, w) + b

        if name=="out":
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)

        tf.compat.v1.summary.histogram("weights", w)
        tf.compat.v1.summary.histogram("bias", b)
        return layer_out

In [695]:
# Model without Dropout

# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name="layer_1")
# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name="layer_2")
# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[NR_CLASSES], name="out")

# model_name = f"{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}"

In [696]:
#Model with Dropout layer

layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name="layer_1")

layer_drop = tf.nn.dropout(layer_1, rate=0.2, name="dropout_layer")

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name="layer_2")
output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[NR_CLASSES], name="out")

model_name = f"{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}"

# Tensorboard Setup

In [698]:
# Folder for Tensorboard

folder_name = f"{model_name} at {strftime('%H%M')}"
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created directories!")

Successfully created directories!


# Loss, Optimization & Metrics

#### Defining Loss function

In [701]:
with tf.name_scope("loss_calc"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

#### Defining Optimizer

In [703]:
with tf.name_scope("optimizer"):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [705]:
with tf.name_scope("accuracy_cal"):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [706]:
with tf.name_scope("performance"):
    tf.compat.v1.summary.scalar("accuracy", accuracy)
    tf.compat.v1.summary.scalar("cost", loss)

#### Check Input Images in Tensorboard

In [708]:
with tf.name_scope("show_image"):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.compat.v1.summary.image("image_input", x_image, max_outputs=4)

# Run Session

In [710]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merge Summaries

In [712]:
merged_summary = tf.compat.v1.summary.merge_all()

train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

train_writer.add_graph(sess.graph)

In [713]:
# Initialise all the variables
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

## Batching the Data

In [715]:
size_of_batch = 1000

In [716]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples / size_of_batch)

index_in_epoch = 0

In [717]:
def next_batch(batch_size, data, labels):

    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += size_of_batch

    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch

    return data[start:end], labels[start:end]

# Training Loop

In [719]:
for epoch in range(nr_epochs):
    
    # ================== Training =========================
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)

    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)

    train_writer.add_summary(s, epoch)
    
    print(f"Epoch {epoch} \t| Training Accuracy = {batch_accuracy}")

    # ======================== Validation ==============================

    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
    
print("Done Training!")

Epoch 0 	| Training Accuracy = 0.8539999723434448
Epoch 1 	| Training Accuracy = 0.8600000143051147
Epoch 2 	| Training Accuracy = 0.8679999709129333
Epoch 3 	| Training Accuracy = 0.8700000047683716
Epoch 4 	| Training Accuracy = 0.8790000081062317
Epoch 5 	| Training Accuracy = 0.8759999871253967
Epoch 6 	| Training Accuracy = 0.8759999871253967
Epoch 7 	| Training Accuracy = 0.9750000238418579
Epoch 8 	| Training Accuracy = 0.9739999771118164
Epoch 9 	| Training Accuracy = 0.9819999933242798
Epoch 10 	| Training Accuracy = 0.9850000143051147
Epoch 11 	| Training Accuracy = 0.984000027179718
Epoch 12 	| Training Accuracy = 0.9850000143051147
Epoch 13 	| Training Accuracy = 0.9860000014305115
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9890000224113464
Epoch 17 	| Training Accuracy = 0.9919999837875366
Epoch 18 	| Training Accuracy = 0.9879999756813049
Epoch 19 	| Training Accuracy = 0.99099999

# Make a Prediction

In [721]:
img = Image.open("MNIST/test_img.png")
img

In [722]:
bw = img.convert("L")

In [723]:
img_array = np.invert(bw)

In [724]:
img_array.shape

(28, 28)

In [725]:
test_img = img_array.ravel()

In [726]:
test_img.shape

(784,)

In [727]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [728]:
print(f"Prediction for test image is {prediction}")

Prediction for test image is [2]


# Testing and Evaluation

In [731]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f"Accuracy on test set is {test_accuracy:0.2%}")

Accuracy on test set is 97.74%


# Reset for the Next Run

In [734]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()

# Code for 1st Part of Module

In [ ]:
# with tf.name_scope("hidden_1"):
#     #Initial weights
#     initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name="w1")
#     #Initial bias
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name="b1")
#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [ ]:
# #Second hidden layer
# with tf.name_scope("hidden_2"):
#     initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name="w2")
    
#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name="b2")
    
#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [ ]:
# # Output layer
# with tf.name_scope("otuput_layer"):
#     initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name="w3")
    
#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name="b3")
    
#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [ ]:
# w1.eval(sess)